Reading in the data and then filtering out SNPs which do not meet p value significance before joining the data together.

In [1]:
%load_ext autoreload
%autoreload 2

import polars as pl

# exp: Exposure
# out: Outcome
# ea: Exposure allele
# oa: Other allele

exp_header_dict = {
  'rsID':'rsid',
  'CHROM':'chr_exp',
  'ALT':'ea_exp',
  'REF':'oa_exp',
  'POOLED_ALT_AF':'eaf_exp',
  'EFFECT_SIZE':'beta_exp',
  'SE':'se_exp',
  'pvalue':'pval_exp'
}

out_header_dict = {
  'markername':'rsid',
  'chr':'chr_out',
  'bp_hg19':'pos_out',
  'effect_allele':'ea_out',
  'noneffect_allele':'oa_out',
  'effect_allele_freq':'eaf_out',
  'beta':'beta_out',
  'se_dgc':'se_out',
  'p_dgc':'pval_out'}

pthresh = 5e-8

# Renaming columns and filtering data to only include observations which fulfill significance threshold
dexp = (pl.scan_csv("dataset/ldlc_gwas.txt",separator="\t")
        .rename(exp_header_dict)
        .filter((pl.col('pval_exp') < pthresh)))
dout = (pl.scan_csv("dataset/mi_gwas.tsv",separator="\t")
        .rename(out_header_dict))

combined = (dexp.join(dout, on='rsid')
            # Convert all data to lowercase
            .with_columns(
                pl.col('ea_exp').str.to_lowercase(),
                pl.col('oa_exp').str.to_lowercase(),
                pl.col('ea_out').str.to_lowercase(),
                pl.col('oa_out').str.to_lowercase(),
            # Convert minor allele freq to effect allele freq
                pl.col('eaf_exp').mul(-1).add(1))
            .collect())

print(combined.shape)

(75089, 26)


We can use the provided harmonize function.

In [2]:
from MR.harmonize import harmonize

total = harmonize(combined, palindromic_action=1, palindromic_threshold=0.08)
print(total.shape)

# Seeing if duplicates are dropped
print(total.unique(subset=['rsid']).shape)

(73095, 26)
(73095, 26)


Clumping the data based on Linkage Disequilibirum (LD). This is to filter out SNPs which are closely correlated in order to essentially avoid double counting results into the final calculation.

In [3]:
from MR.ld import ld_clump

pruned_rsids = ld_clump(total['rsid'], total['pval_exp'])

processed_data = (total.join(pruned_rsids, on='rsid'))

print(processed_data.shape)

Calculating causal effects.

In [9]:
from MR.calculate_effect import calculate_effect, methods

print(processed_data.shape)
print(methods)
print()

# print('MR Presso')
# result = calculate_effect(processed_data, 'presso')
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}\n')

# print('Inverse Variance Weighted')
# result = calculate_effect(processed_data, 'inverse_variance_weighted')
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Wald ratio')
# result = calculate_effect(processed_data, 'wald_ratio')
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Simple Median')
# result = calculate_effect(data=processed_data, method='simple_median')
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Weighted Median')
# result = calculate_effect(processed_data, 'weighted_median')
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Penalised Weighted Median')
# result = calculate_effect(processed_data, 'penalised_weighted_median')
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Egger Regression')
# result = calculate_effect(processed_data, 'egger_regression')
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Simple Mode')
# result = calculate_effect(processed_data, 'simple_mode')
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Weighted Mode')
# result = calculate_effect(processed_data, 'weighted_mode')
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# # print('Penalised Weighted Mode')
# # result = calculate_effect(processed_data, 'penalised_weighted_mode')
# # print(f'Effect: {result["effect"]}')
# # print(f'se: {result["se"]}')
# # print(f'pval: {result["pval"]}\n')

# print('Maximum Likelihood')
# result = calculate_effect(processed_data, 'maximum_likelihood')
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')


(379, 26)
['inverse_variance_weighted', 'wald_ratio', 'maximum_likelihood', 'simple_median', 'weighted_median', 'penalised_weighted_median', 'simple_mode', 'weighted_mode', 'penalised_weighted_mode', 'egger_regression', 'presso']

Simple Median
Effect: 0.5134897781385475
se: 0.05539473337724837
pval: 1.8675812265736418e-20

Weighted Median
Effect: 2.253737117423882
se: 0.14443006375045928
pval: 6.799982962471704e-55

Penalised Weighted Median
Effect: 2.2544937754237173
se: 0.14870361236164836
pval: 6.409720285565536e-52



In [10]:
# print('MR Presso')
# result = calculate_effect(processed_data, 'presso')
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}\n')

from MR.mr_methods.egger_regression import mr_egger_regression
from MR.mr_methods.inverse_variance_weighted import mr_inverse_variance_weighted
from MR.mr_methods.maximum_likelihood import mr_maximum_likelihood
from MR.mr_methods.median import mr_penalised_weighted_median, mr_simple_median, mr_weighted_median
from MR.mr_methods.mode import mr_simple_mode, mr_weighted_mode
from MR.mr_methods.wald_ratio import mr_wald_ratio

data = processed_data

print('Inverse Variance Weighted')
result = mr_inverse_variance_weighted(data['beta_exp'], data['beta_out'], data['se_out'])
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}')
print(f'pval: {result["pval"]}\n')

print('Wald ratio')
result = mr_wald_ratio(data['beta_exp'], data['beta_out'], data['se_out'])
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}')
print(f'pval: {result["pval"]}\n')

print('Simple Median')
result = mr_simple_median(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'], 1000)
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}')
print(f'pval: {result["pval"]}\n')

print('Weighted Median')
result = mr_weighted_median(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}')
print(f'pval: {result["pval"]}\n')

print('Penalised Weighted Median')
result = mr_penalised_weighted_median(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}')
print(f'pval: {result["pval"]}\n')

print('Egger Regression')
result = mr_egger_regression(data['beta_exp'], data['beta_out'], data['se_out'])
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}')
print(f'pval: {result["pval"]}\n')

print('Simple Mode')
result = mr_simple_mode(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}')
print(f'pval: {result["pval"]}\n')

print('Weighted Mode')
result = mr_weighted_mode(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}')
print(f'pval: {result["pval"]}\n')

# print('Penalised Weighted Mode')
# result = mr_penalised_weighted_mode(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

print('Maximum Likelihood')
result = mr_maximum_likelihood(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
print(f'Effect: {result["effect"]}')
print(f'se: {result["se"]}')
print(f'pval: {result["pval"]}\n')


Inverse Variance Weighted
Effect: 0.4129020632417699
se: 0.04660832025520153
pval: 0.0

